<a href="https://colab.research.google.com/github/dkswndms4782/IGPS_project/blob/master/MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [5]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [6]:
#MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [7]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [8]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.keep_prob = 0.5

    self.layer1 = nn.Sequential(
        nn.Conv2d(1,32,kernel_size = 3,stride =1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32,64,kernel_size = 3,stride = 1,padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,stride = 2)
    )
    self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
    self.fc1 = nn.Linear(4*4*128,625,bias = True)
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    self.layer4 = nn.Sequential(
        self.fc1,
        nn.ReLU(),
        nn.Dropout(p = 1 - self.keep_prob))
    self.fc2 = nn.Linear(625,10,bias = True)
    nn.init.xavier_uniform_(self.fc2.weight)
    

  def forward(self,x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0),-1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [9]:
  model = CNN().to(device)

In [10]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [11]:
total_batch = len(data_loader)

In [ ]:
for epoch in range(training_epochs):
  avg_cost = 0
  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)

    cost = criterion(hypothesis,Y)
    cost.backward()
    optimizer.step()
    
    avg_cost += cost / total_batch
  print('[Epoch : {}] cost = {}'.format(epoch+1,avg_cost))
print('Learning Finished!')

In [ ]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction,1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:',accuracy.item())